In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import copy
import time
import cv2
import numpy as np
import random

from copy import deepcopy 
from collections import deque, defaultdict
from typing import Any, Dict, List, Optional, Set, Tuple, Union
from numpy.typing import ArrayLike, NDArray

import dm_control
from dm_control import mujoco as dm_mujoco
from dm_control.utils.transformations import mat_to_quat, quat_to_euler, euler_to_quat
import mujoco
from rocobench.envs import SortOneBlockTask, CabinetTask, MoveRopeTask, SweepTask, MakeSandwichTask, PackGroceryTask, MujocoSimEnv, SimRobot, visualize_voxel_scene

# use conda environment roco4 for this notebook

# Render the scene
We start the sim environment, and render the set of cameras that was added specifically to get full coverage of the environment.

USE Roco4

In [ ]:
tracked = [
    'sceneshotcam', 
    'apple', 
    'milk', 
    'cereal', 
    'bread', 
    'banana', 
    'bin', 
    'ur5e', 
    'panda', 
    'table',
    'center_top_cam',
    'genpurp__1',
    'genpurp__2',
    'genpurp__3',
    'genpurp__4',
    'genpurp__5',
    'genpurp__6',
    'genpurp__7',
]
# tracked = ['rotcam0', 'rotcam1', 'rotcam2', 'rotcam3', 'rotcam4', 'rotcam5', 'rotcam6', 'rotcam7', 'rotcam8', 'rotcam9', 'rotcam10', 'rotcam11', 'rotcam12', 'rotcam13', 'rotcam14', 'rotcam15', 'rotcam16', 'rotcam17', 'rotcam18', 'rotcam19', 'rotcam20', 'rotcam21', 'rotcam22', 'rotcam23', 'rotcam24', 'rotcam25', 'rotcam26', 'rotcam27', 'rotcam28', 'rotcam29', 'rotcam30', 'rotcam31', 'rotcam32', 'rotcam33', 'rotcam34', 'rotcam35', 'rotcam36', 'rotcam37', 'rotcam38', 'rotcam39', 'rotcam40', 'rotcam41', 'rotcam42', 'rotcam43', 'rotcam44', 'rotcam45', 'rotcam46', 'rotcam47', 'rotcam48', 'rotcam49', 'rotcam50', 'rotcam51', 'rotcam52', 'rotcam53', 'rotcam54', 'rotcam55', 'rotcam56', 'rotcam57', 'rotcam58', 'rotcam59', 'rotcam60', 'rotcam61', 'rotcam62', 'rotcam63', 'rotcam64', 'rotcam65', 'rotcam66', 'rotcam67', 'rotcam68', 'rotcam69', 'rotcam70', 'rotcam71', 'rotcam72', 'rotcam73', 'rotcam74', 'rotcam75', 'rotcam76', 'rotcam77', 'rotcam78', 'rotcam79', 'rotcam80', 'rotcam81', 'rotcam82', 'rotcam83', 'rotcam84', 'rotcam85', 'rotcam86', 'rotcam87', 'rotcam88', 'rotcam89', 'rotcam90', 'rotcam91', 'rotcam92', 'rotcam93', 'rotcam94', 'rotcam95', 'rotcam96', 'rotcam97', 'rotcam98', 'rotcam99', 'rotcam100', 'rotcam101', 'rotcam102', 'rotcam103', 'rotcam104', 'rotcam105', 'rotcam106', 'rotcam107', 'rotcam108', 'rotcam109', 'rotcam110', 'rotcam111', 'rotcam112', 'rotcam113', 'rotcam114', 'rotcam115', 'rotcam116', 'rotcam117', 'rotcam118', 'rotcam119', 'rotcam120', 'rotcam121', 'rotcam122', 'rotcam123', 'rotcam124', 'rotcam125', 'rotcam126', 'rotcam127', 'rotcam128', 'rotcam129', 'rotcam130', 'rotcam131', 'rotcam132', 'rotcam133', 'rotcam134', 'rotcam135', 'rotcam136', 'rotcam137', 'rotcam138', 'rotcam139', 'rotcam140', 'rotcam141', 'rotcam142', 'rotcam143', 'rotcam144', 'rotcam145', 'rotcam146', 'rotcam147', 'rotcam148', 'rotcam149', 'rotcam150', 'rotcam151', 'rotcam152', 'rotcam153', 'rotcam154', 'rotcam155', 'rotcam156', 'rotcam157', 'rotcam158', 'rotcam159', 'rotcam160', 'rotcam161', 'rotcam162', 'rotcam163', 'rotcam164', 'rotcam165', 'rotcam166', 'rotcam167', 'rotcam168', 'rotcam169', 'rotcam170', 'rotcam171', 'rotcam172', 'rotcam173', 'rotcam174', 'rotcam175', 'rotcam176', 'rotcam177', 'rotcam178', 'rotcam179', 'rotcam180', 'rotcam181', 'rotcam182', 'rotcam183', 'rotcam184', 'rotcam185', 'rotcam186', 'rotcam187', 'rotcam188', 'rotcam189', 'rotcam190', 'rotcam191', 'rotcam192', 'rotcam193', 'rotcam194', 'rotcam195', 'rotcam196', 'rotcam197', 'rotcam198', 'rotcam199', 'rotcam200', 'rotcam201', 'rotcam202', 'rotcam203', 'rotcam204', 'rotcam205', 'rotcam206', 'rotcam207', 'rotcam208', 'rotcam209', 'rotcam210', 'rotcam211', 'rotcam212', 'rotcam213', 'rotcam214', 'rotcam215', 'rotcam216', 'rotcam217', 'rotcam218', 'rotcam219', 'rotcam220', 'rotcam221', 'rotcam222', 'rotcam223', 'rotcam224', 'rotcam225', 'rotcam226', 'rotcam227', 'rotcam228', 'rotcam229', 'rotcam230', 'rotcam231', 'rotcam232', 'rotcam233', 'rotcam234', 'rotcam235', 'rotcam236']
# tracked = ['rotcam1']

env = PackGroceryTask(
    render_freq=2000,
    image_hw=(512,512),
    sim_forward_steps=300,
    error_freq=30,
    error_threshold=1e-5,
    randomize_init=True,
    render_point_cloud=0,
    render_cameras=["face_panda","face_ur5e","teaser",],
    point_feature_cameras=tracked,
    one_obj_each=True,
)

In [ ]:
# to change the depth camera resolution, change the znear parameter in line 880 of (default value is 0.01, but higher works for conceptfusion)
# /opt/conda/envs/roco4/lib/python3.8/site-packages/dm_control/mujoco/engine.py

env.physics.forward()
outputs = env.render_feature_cameras() # I added this method to the class, the file is in /rocobench/envs/base_env.py if you wish to see the code

# Save the images captured by the feature cameras
Here we render all of the cameras that are used for this purpose, but we only use ones tracking objects for object features

In [ ]:
from PIL import Image
for cam in tracked:
    rgbimg = Image.fromarray(outputs[cam][1])
    rgbimg.save(f'feature_pictures/{cam}.png')

Next we plot the points to verify that there are no artifacts and to view the object segementation results.

In [ ]:
from time import time
start_pointcloudagg_time = time()
SCENE_BOUNDS=((-1.4, -0.2, -0.1), (1.7, 1.2, 1.1)) # taken from RoCo paper, based on the scene bounds of the PackGroceryTask
# SCENE_BOUNDS=((-40, -40, -40), (40, 40, 40)) # taken from RoCo paper, based on the scene bounds of the PackGroceryTask
point_clouds = [
    sensor_output[0].point_cloud.filter_bounds(bounds=SCENE_BOUNDS) 
        for sensor_output in outputs.values()
]
global_point_cloud = sum(point_clouds[1:], start=point_clouds[0])
end_pointcloudagg_time = time()
print(f"took {end_pointcloudagg_time - start_pointcloudagg_time} seconds to aggregate point clouds")

In [ ]:
print(outputs)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d

def plot_class_cloud(point_cloud, excluding=None, segmented=False):
    fig = plt.figure(figsize=(6, 6), dpi=160)
    ax = fig.add_subplot(111, projection="3d")
    background_color = (1.0, 1.0, 1.0)
    ax.set_facecolor(background_color)
    ax.w_xaxis.set_pane_color(background_color)  # type: ignore
    ax.w_yaxis.set_pane_color(background_color)  # type: ignore
    ax.w_zaxis.set_pane_color(background_color)  # type: ignore
    point_cloud = point_cloud.filter_bounds(bounds=SCENE_BOUNDS)

    xyz_pts = point_cloud.xyz_pts
    rgb_pts = point_cloud.rgb_pts/255.0

    # optionally exclude certain classes
    mask = np.ones(len(point_cloud), dtype=bool)
    if excluding is not None:
        for key, value in point_cloud.segmentation_pts.items():
            if key in excluding and np.any(value):
                mask &= ~value

        xyz_pts = xyz_pts[mask]
        rgb_pts = rgb_pts[mask]

    # changes the colors of the point cloud to correspond to object classes
    if segmented:
        num_classes = len(point_cloud.segmentation_pts.keys())
        colors = sns.color_palette("hsv", num_classes)
        for i, (key, value) in enumerate(point_cloud.segmentation_pts.items()):
            rgb_pts[value] = colors[i]

    x, y, z = (
        xyz_pts[mask, 0],
        xyz_pts[mask, 1],
        xyz_pts[mask, 2],
    )

    ax.scatter(
        x,
        y,
        z,
        c=rgb_pts.astype(float),
        s=pts_size,  # type: ignore
    )
    # ax.set_xticks([])
    # ax.set_yticks([])
    # ax.set_zticks([]) 
    ax.axes.set_xlim3d(left=SCENE_BOUNDS[0][0], right=SCENE_BOUNDS[1][0])  # type: ignore
    ax.axes.set_ylim3d(bottom=SCENE_BOUNDS[0][1], top=SCENE_BOUNDS[1][1])  # type: ignore
    ax.axes.set_zlim3d(bottom=SCENE_BOUNDS[0][2], top=SCENE_BOUNDS[1][2])  # type: ignore
    # plt.tight_layout(pad=0)
    # plt.show()

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(xyz_pts)
    point_cloud.colors = o3d.utility.Vector3dVector(rgb_pts)

    # Save the point cloud to a .ply file
    o3d.io.write_point_cloud("rotated_point_cloud_segments.ply", point_cloud)

pts_size = 0.1
plot_class_cloud(global_point_cloud, excluding=None, segmented=True)

- Finally, we save the environment observation to use later.

In [ ]:
from PIL import Image
import cv2
new_output = {}
for key in outputs:
    # if key == "ur5e" or key == "panda": # these two cameras ended up sideways with how I had to orient them to capture the robotic arms
    #     img = cv2.rotate(outputs[key][1], cv2.ROTATE_90_COUNTERCLOCKWISE)
    #     depth_img = cv2.rotate(outputs[key][2], cv2.ROTATE_90_COUNTERCLOCKWISE)
    # else:
    img = outputs[key][1]
    depth_img = outputs[key][2]
        
    pose = outputs[key][0].extrinsic
    print(key)
    print(outputs[key][0].intrinsic)
    print(outputs[key][0].get_camera_matrix())
    new_output[key] = (img, depth_img, pose)

# coming out of mujoco, the points are usually very small (on the order of [0,2]), which causes them to almost all become 0 when casting to long
# we scale up the values by a factor of 100 since empirically that proved to be a good scale in terms of number of points preserveed
# after converting to long without the dimensions starting to become too large
scaled_xyz_pts = global_point_cloud.xyz_pts * 100

new_output['xyz_pts'] = scaled_xyz_pts
new_output['segmentation_pts'] = global_point_cloud.segmentation_pts
new_output['rgb_pts'] = global_point_cloud.rgb_pts

import pickle

with open('outputs.pkl', 'wb') as f:
    pickle.dump(new_output, f)

In [ ]:
# also save in format usable by concept fusion
from PIL import Image

# with open("outputs.pkl", "rb") as f:
#     outputs = pickle.load(f)

for key in outputs:
    with open(f'simulator_dataset/r1_lab/rgb/{key}.png', 'wb') as f:
        img = Image.fromarray(outputs[key][1])
        img.save(f)
    
    with open(f'simulator_dataset/r1_lab/depth/{key}.png', 'wb') as f:
        img = Image.fromarray(outputs[key][2])
        img.save(f)

    with open(f'simulator_dataset/r1_lab/poses/{key}.np', 'wb') as f:
        np.save(f, outputs[key][0].extrinsic)

In [ ]:
cams = []
cam_commands = []
cam_num = 0
for i in range(725, 395, -5):
    cam_command = f"""<camera mode="fixed" name='rotcam{cam_num}' pos="-1.000 0.500 0.{i}" xyaxes="-0.482 -0.000 -0.876 0.000 -1.000 0.000"/>"""
    cam_commands.append(cam_command)
    cams.append(f'rotcam{cam_num}')
    cam_num += 1

for i in range(475, 95, -5):
    cam_command = f"""<camera mode="fixed" name='rotcam{cam_num}' pos="-1.000 0.{i} 0.400" xyaxes="-0.482 -0.000 -0.876 0.000 -1.000 0.000"/>"""
    cam_commands.append(cam_command)
    cams.append(f'rotcam{cam_num}')
    cam_num += 1

for i in range(525, 1000, 5):
    cam_command = f"""<camera mode="fixed" name='rotcam{cam_num}' pos="-1.000 0.{i} 0.400" xyaxes="-0.482 -0.000 -0.876 0.000 -1.000 0.000"/>"""
    cam_commands.append(cam_command)
    cams.append(f'rotcam{cam_num}')
    cam_num += 1

print(cams)
print("\n".join(cam_commands))

conda activate 3dllm

python extract_conceptfusion_features.py --dataconfig_path="dataconfigs/simulator.yaml" --data_dir="data/simulator_dataset" --sequence="r1_lab" --checkpoint_path="checkpoints/sam_vit_h_4b8939.pth" --stride=1

python run_feature_fusion_and_save_map.py --dataconfig_path="dataconfigs/simulator.yaml" --dataset_path="data/simulator_dataset" --sequence="r1_lab" --checkpoint_path="checkpoints/sam_vit_h_4b8939.pth" --stride=1 --image_height=512 --image_width=512 --desired_feature_height=512 --desired_feature_width=512